In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt

Loading datas from CryptoPredictor and StockPredictor files

In [28]:
# Load Model
stock_model = tf.keras.models.load_model('stock_predict')
crypto_model = tf.keras.models.load_model('crypto_predict')

# Load Data
df = pd.read_csv('predict_data.csv')
stock_column = ['VN-Index', 'S&P 500', 'STOXX', 'VND/USD',
                'VND/EUR', 'interest rate', 'inflation', 'GDP']
crypto_column = ['SS', 'GSPC', 'N225', '^STOXX', 'Index']

# Load Scaler
stock_data_scaler = joblib.load('stock_data_scaler.save')
crypto_data_scaler = joblib.load('crypto_data_scaler.save')
stock_predict_scaler = joblib.load('stock_predict_scaler.save')
crypto_predict_scaler = joblib.load('crypto_predict_scaler.save')


Joining Savings, Stock, and Crypto data

In [29]:
# Transform Input Data
stock_data = stock_data_scaler.transform(df[stock_column].values)
crypto_data = crypto_data_scaler.transform(df[crypto_column].values)

# Predict
predicted_stock = stock_model.predict(np.reshape(
    stock_data, (1, stock_data.shape[0], stock_data.shape[1])))
predicted_stock = np.reshape(predicted_stock, (-1, 1))
predicted_stock = stock_predict_scaler.inverse_transform(predicted_stock)

predicted_crypto = crypto_model.predict(np.reshape(
    crypto_data, (1, crypto_data.shape[0], crypto_data.shape[1])))
predicted_crypto = np.reshape(predicted_crypto, (-1, 1))
predicted_crypto = crypto_predict_scaler.inverse_transform(predicted_crypto)

predicted_saving = df['Saving'][:40]


1/1 [==============================] - 0s 61ms/step


In [30]:
# Create DataFrame for Portfolio Allocation
df = pd.DataFrame(predicted_saving)
df = df.join(pd.DataFrame(predicted_stock))
df = df.join(pd.DataFrame(predicted_crypto), lsuffix='_left', rsuffix='_right')
df.columns = ['Saving', 'VN-Index', 'Crypto']

Modern Portfolio Theory

In [31]:
cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()
corr_matrix = df.pct_change().apply(lambda x: np.log(1+x)).corr()
ann_sd = df.pct_change().apply(lambda x: np.log(
    1+x)).std().apply(lambda x: x*np.sqrt(250))
ind_er = df.pct_change().sum()

In [32]:
p_ret = []  # Define an empty array for portfolio returns
p_vol = []  # Define an empty array for portfolio volatility
p_weights = []  # Define an empty array for asset weights

num_assets = len(df.columns)
num_portfolios = 10000

for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    # Returns are the product of individual expected returns of asset and its weights
    returns = np.dot(weights, ind_er)
    p_ret.append(returns)

    var = cov_matrix.mul(weights, axis=0).mul(
        weights, axis=1).sum().sum()  # Portfolio Variance
    sd = np.sqrt(var)  # Daily standard deviation
    ann_sd = sd*np.sqrt(250)  # Annual standard deviation = volatility
    p_vol.append(ann_sd)

data = {'Returns': p_ret, 'Volatility': p_vol}

for counter, symbol in enumerate(['savings', 'VN-Index', 'Crypto']):
    data[symbol+' weight'] = [w[counter] for w in p_weights]

portfolios = pd.DataFrame(data)

In [33]:
# Finding the optimal portfolio
rf = .05  # risk factor that could change based on investor's taste
optimal_risky_port = portfolios.iloc[(
    (portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
optimal_risky_port

Returns           -0.113863
Volatility         1.643828
savings weight     0.074540
VN-Index weight    0.001084
Crypto weight      0.924376
Name: 9521, dtype: float64